### retrieve 기능

In [ ]:
import os

os.environ["GROQ_API_KEY"]=""
os.environ["TAVILY_API_KEY"]=""
os.environ["GOOGLE_API_KEY"]=""

In [67]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI

embeddings = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001"
)

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash"
)

In [ ]:
!pip install langchain_google_genai

In [69]:
urls = [
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
]

In [70]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma

#3개의 사이트에서 가져온 문서를 chunck 단위로 나눈것을 list

docs = [WebBaseLoader(url).load() for url in urls]
docs_list =[item for sublist in docs for item in sublist]
# docs_list

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=250, chunk_overlap=10
)

doc_splits = text_splitter.split_documents(docs_list)

vectorstore = Chroma.from_documents(
    documents=doc_splits,
    embedding=embeddings,
    collection_name="langgraph",
    persist_directory="/content/drive/MyDrive/DB/adative_rag_db"
)

retriever = vectorstore.as_retriever()

In [ ]:
!pip install langchain langchain_community chromadb

### query grade

In [72]:
### Router

from typing import Literal

from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

from pydantic import BaseModel, Field


# Data model
class RouteQuery(BaseModel):
    """Route a user query to the most relevant datasource."""

    datasource: Literal["vectorstore", "web_search"] = Field(
        ...,
        description="Given a user question choose to route it to web search or a vectorstore.",
    )


# LLM with function call
# llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash",
                             model_kwargs={"streaming": True},
                             convert_system_message_to_human=True,       # 사용할 Gemini 모델 이름 (예: gemini-pro, gemini-2.0-flash 등)
                            temperature=0         # 선택적 파라미터: 응답 다양성 조절
)
structured_llm_router = llm.with_structured_output(RouteQuery)

# Prompt
system_prompt = """You are an expert at routing a user question to a vectorstore or web search.
The vectorstore contains documents related to agents, prompt engineering, and adversarial attacks.
Use the vectorstore for questions on these topics. Otherwise, use web-search."""
route_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{question}"),
    ]
)
print(route_prompt)
question_router = route_prompt | structured_llm_router

input_variables=['question'] input_types={} partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are an expert at routing a user question to a vectorstore or web search.\nThe vectorstore contains documents related to agents, prompt engineering, and adversarial attacks.\nUse the vectorstore for questions on these topics. Otherwise, use web-search.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})]


In [73]:
question_router.invoke(
    {"question": "Who will the Bears draft first in the NFL draft?"}
)

/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:390: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


RouteQuery(datasource='web_search')

In [74]:
result = question_router.invoke(
    {"question": "What are the types of agent memory?"}
)
result

/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:390: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


RouteQuery(datasource='vectorstore')

In [75]:
if result.datasource == "web_search":
  print("process web search")
else:
  print("리트리브 기능 수행")

리트리브 기능 수행


### generate 기능

In [76]:
### Router

from typing import Literal

from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

from pydantic import BaseModel, Field
from langchain import hub
from langchain_core.output_parsers import StrOutputParser

# LLM with function call
# llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash",
                             model_kwargs={"streaming": True},
                             convert_system_message_to_human=True,       # 사용할 Gemini 모델 이름 (예: gemini-pro, gemini-2.0-flash 등)
                            temperature=0         # 선택적 파라미터: 응답 다양성 조절
)


# Prompt
generate_prompt = hub.pull("rlm/rag-prompt")
print(generate_prompt)

rag_chain = generate_prompt | llm | StrOutputParser()

input_variables=['context', 'question'] input_types={} partial_variables={} metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})]


/usr/local/lib/python3.11/dist-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [77]:
rag_chain.invoke({"context":["""Key Capabilities
Code When You Need It: Write JavaScript/Python, add npm packages, or use the visual interface
AI-Native Platform: Build AI agent workflows based on LangChain with your own data and models
Full Control: Self-host with our fair-code license or use our cloud offering
Enterprise-Ready: Advanced permissions, SSO, and air-gapped deployments
Active Community: 400+ integrations and 900+ ready-to-use templates"""], "question":"i am boy"})

/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:390: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


'I am an AI assistant and cannot determine your gender based on the provided context. The context describes the capabilities of a platform for building AI agent workflows. It mentions features like code writing, AI-native platform, full control, enterprise readiness, and an active community.'

### documents relevant question 기능

In [78]:
### Retrieval Grader


# Data model
class GradeDocuments(BaseModel):
    """Binary score for relevance check on retrieved documents."""

    binary_score: str = Field(
        description="Documents are relevant to the question, 'yes' or 'no'"
    )


# LLM with function call
# llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash",
                             model_kwargs={"streaming": True},
                             convert_system_message_to_human=True,       # 사용할 Gemini 모델 이름 (예: gemini-pro, gemini-2.0-flash 등)
                              temperature=0.7         # 선택적 파라미터: 응답 다양성 조절
)
structured_llm_grader = llm.with_structured_output(GradeDocuments)

# Prompt
system = """You are a grader assessing relevance of a retrieved document to a user question. \n
    If the document contains keyword(s) or semantic meaning related to the user question, grade it as relevant. \n
    It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question."""
grade_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Retrieved document: \n\n {document} \n\n User question: {question}"),
    ]
)

retrieval_grader = grade_prompt | structured_llm_grader

In [79]:
retrieved_docs = retriever.invoke("tell me about seoul")[0].page_content
retrieval_grader.invoke({"question": "tell me about seoul", "document":retrieved_docs})

/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:390: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


GradeDocuments(binary_score='yes')

'Later, Shum et al. (2023) found that in their experiments CoT prompts with only complex examples can improve the accuracy of complex questions, but perform poorly in simple questions; evidence shown on GSM8k.\n\n\nChanging Q: to Question: is found to be helpful. (Fu et al. 2023)\n\n\nYe & Durrett (2022) found that the benefit of including explanations in the prompt is small to moderate for NLP tasks that involve reasoning over text (i.e. QA and NLI) and the effects vary by models. They observed that explanations are more likely to be nonfactual than be inconsistent (i.e. whether explanation entails prediction). Nonfactual explanations most likely lead to incorrect predictions.'

### 질문 재조합 기능

In [89]:
### Question Re-writer

# LLM
# llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash",
                             model_kwargs={"streaming": True},
                             convert_system_message_to_human=True,       # 사용할 Gemini 모델 이름 (예: gemini-pro, gemini-2.0-flash 등)
                              temperature=0         # 선택적 파라미터: 응답 다양성 조절
)
# Prompt
system = """You are a question re-writer that converts an input question to a better version that is optimized \n
     for vectorstore retrieval. Look at the input and try to reason about the underlying semantic intent / meaning."""
re_write_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        (
            "human",
            "Here is the initial question: \n\n {question} \n Formulate an improved question.",
        ),
    ]
)

question_rewriter = re_write_prompt | llm | StrOutputParser()

### Hallucination 판단기능

In [80]:
### Hallucination Grader


# Data model
class GradeHallucinations(BaseModel):
    """Binary score for hallucination present in generation answer."""

    binary_score: str = Field(
        description="Answer is grounded in the facts, 'yes' or 'no'"
    )


# LLM with function call
# llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash",
                             model_kwargs={"streaming": True},
                             convert_system_message_to_human=True,       # 사용할 Gemini 모델 이름 (예: gemini-pro, gemini-2.0-flash 등)
)
structured_llm_grader = llm.with_structured_output(GradeHallucinations)

# Prompt
system = """You are a grader assessing whether an LLM generation is grounded in / supported by a set of retrieved facts. \n
     Give a binary score 'yes' or 'no'. 'Yes' means that the answer is grounded in / supported by the set of facts."""
hallucination_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Set of facts: \n\n {documents} \n\n LLM generation: {generation}"),
    ]
)

hallucination_grader = hallucination_prompt | structured_llm_grader

In [65]:
retrieved_docs = retriever.invoke("tell me about LLM ")[0].page_content
hallucination_grader.invoke({"generation": "tell me about LLM", "documents":retrieved_docs})

/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:390: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


GradeHallucinations(binary_score='no')

### answer와 question 적정성 기능

In [81]:
### Answer Grader


# Data model
class GradeAnswer(BaseModel):
    """Binary score to assess answer addresses question."""

    binary_score: str = Field(
        description="Answer addresses the question, 'yes' or 'no'"
    )


# LLM with function call
# llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash",
                             model_kwargs={"streaming": True},
                             convert_system_message_to_human=True,       # 사용할 Gemini 모델 이름 (예: gemini-pro, gemini-2.0-flash 등)
                              temperature=0         # 선택적 파라미터: 응답 다양성 조절
)
structured_llm_grader = llm.with_structured_output(GradeAnswer)

# Prompt
system = """You are a grader assessing whether an answer addresses / resolves a question \n
     Give a binary score 'yes' or 'no'. Yes' means that the answer resolves the question."""
answer_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "User question: \n\n {question} \n\n LLM generation: {generation}"),
    ]
)

answer_grader = answer_prompt | structured_llm_grader

### web search 기능


In [82]:
### Search

from langchain_community.tools.tavily_search import TavilySearchResults

web_search_tool = TavilySearchResults(k=5)

## 그래프 구조

### State 정의

In [84]:
from typing import List

from typing_extensions import TypedDict , Annotated
from pydantic import BaseModel, Field

class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        question: question
        generation: LLM generation
        documents: list of documents
    """

    question: Annotated[str, ""]
    generation: Annotated[str, ""]
    documents: Annotated[List[str], ["list가들어 간다."]]

### node 함수 만들기

In [85]:
# retrieve node 함수
from langchain.schema import Document


def retrieve(state:GraphState)-> GraphState:
    """
    Retrieve documents

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, documents, that contains retrieved documents
    """
    print("---RETRIEVE---")
    question = state["question"]

    # Retrieval
    documents = retriever.invoke(question)
    return GraphState(documents=documents,question=question)
    # return {"documents": documents, "question": question}

In [86]:
# documents grade 함수
def grade_documents(state:GraphState) -> GraphState:
    """
    Determines whether the retrieved documents are relevant to the question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates documents key with only filtered relevant documents
    """

    print("---CHECK DOCUMENT RELEVANCE TO QUESTION---")
    question = state["question"]
    documents = state["documents"]

    # Score each doc
    filtered_docs = []
    for d in documents:
        score = retrieval_grader.invoke(
            {"question": question, "document": d.page_content}
        )
        grade = score.binary_score
        if grade == "yes":
            print("---GRADE: DOCUMENT RELEVANT---")
            filtered_docs.append(d)
        else:
            print("---GRADE: DOCUMENT NOT RELEVANT---")
            continue
    return {"documents": filtered_docs, "question": question}

In [87]:
# generation 해주는 함수
def generate(state:GraphState) -> GraphState:
    """
    Generate answer

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """
    print("---GENERATE---")
    question = state["question"]
    documents = state["documents"]

    # RAG generation
    generation = rag_chain.invoke({"context": documents, "question": question})
    print(generation)
    return GraphState(documents=documents, question=question, generation=generation)
    # return {"documents": documents, "question": question, "generation": generation

In [90]:
# 질문 재조합하는 함수
def transform_query(state:GraphState) -> GraphState:
    """
    Transform the query to produce a better question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates question key with a re-phrased question
    """

    print("---TRANSFORM QUERY---")
    question = state["question"]
    documents = state["documents"]

    # Re-write question
    better_question = question_rewriter.invoke({"question": question})
    return {"documents": documents, "question": better_question}

In [92]:
# 웹에서 검색하는 node 함수
def web_search(state:GraphState) -> GraphState:
    """
    Web search based on the re-phrased question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates documents key with appended web results
    """

    print("---WEB SEARCH---")
    question = state["question"]

    # Web search
    docs = web_search_tool.invoke({"query": question})
    web_results = "\n".join([d["content"] for d in docs])
    web_results = Document(page_content=web_results)

    return {"documents": web_results, "question": question}

In [93]:
# 질문 index 매칭 판단하는 router 함수
def route_question(state:GraphState) -> str:
    """
    Route question to web search or RAG.

    Args:
        state (dict): The current graph state

    Returns:
        str: Next node to call
    """

    print("---ROUTE QUESTION---")
    question = state["question"]
    source = question_router.invoke({"question": question})
    if source.datasource == "web_search":
        print("---ROUTE QUESTION TO WEB SEARCH---")
        return "web_search"
    elif source.datasource == "vectorstore":
        print("---ROUTE QUESTION TO RAG---")
        return "vectorstore"

In [98]:
# generate로 가는지 판단하는 함수
def decide_to_generate(state:GraphState) -> str:
    """
    Determines whether to generate an answer, or re-generate a question.

    Args:
        state (dict): The current graph state

    Returns:
        str: Binary decision for next node to call
    """

    print("---ASSESS GRADED DOCUMENTS---")
    state["question"]
    filtered_documents = state["documents"]

    if not filtered_documents:
        # All documents have been filtered check_relevance
        # We will re-generate a new query
        print(
            "---DECISION: ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, TRANSFORM QUERY---"
        )
        return "transform_query"
    else:
        # We have relevant documents, so generate answer
        print("---DECISION: GENERATE---")
        return "generate"

In [94]:
# router hallucination and answer is grounded in documents
def grade_generation_v_documents_and_question(state:GraphState) -> str:
    """
    Determines whether the generation is grounded in the document and answers question.

    Args:
        state (dict): The current graph state

    Returns:
        str: Decision for next node to call
    """

    print("---CHECK HALLUCINATIONS---")
    question = state["question"]
    documents = state["documents"]
    generation = state["generation"]

    score = hallucination_grader.invoke(
        {"documents": documents, "generation": generation}
    )
    grade = score.binary_score

    # Check hallucination
    if grade == "yes":
        print("---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---")
        # Check question-answering
        print("---GRADE GENERATION vs QUESTION---")
        score = answer_grader.invoke({"question": question, "generation": generation})
        grade = score.binary_score
        if grade == "yes":
            print("---DECISION: GENERATION ADDRESSES QUESTION---")
            return "useful"
        else:
            print("---DECISION: GENERATION DOES NOT ADDRESS QUESTION---")
            return "not useful"
    else:
        print("---DECISION: GENERATION IS NOT GROUNDED IN DOCUMENTS, RE-TRY---")
        return "not supported"

### graph 만들기

In [99]:
from langgraph.graph import END, StateGraph, START

workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("web_search", web_search)  # web search
workflow.add_node("retrieve", retrieve)  # retrieve
workflow.add_node("grade_documents", grade_documents)  # grade documents
workflow.add_node("generate", generate)  # generate
workflow.add_node("transform_query", transform_query)  # transform_query

workflow.add_conditional_edges(
    START, # 시작 노드 이름
    route_question, # 조건 함수
    {
        "web_search":"web_search",
        "vectorstore":"retrieve" # 결과 키:노드 매핑
    }
)
workflow.add_edge("web_search" , "generate")
workflow.add_edge("retrieve" , "grade_documents")

workflow.add_conditional_edges(
    "grade_documents",
    decide_to_generate,
    {
        "transform_query": "transform_query",
        "generate": "generate",
    },
)

workflow.add_edge("transform_query" , "retrieve")
workflow.add_conditional_edges(
    "generate",
    grade_generation_v_documents_and_question,
    {
        "useful": END,
        "not useful": "transform_query",
        "not supported":"generate"
    },
)

app = workflow.compile()

In [101]:
from pprint import pprint

# Run
# 2024년 NFL 드래프트에서 베어스의 어떤 선수가 먼저 드래프트할 것으로 예상했나요?
# inputs = {
#     "question": "What player at the Bears expected to draft first in the 2024 NFL draft?"
# }

inputs = {"question": "Who will the Bears draft first in the NFL draft?"}

for output in app.stream(inputs):
    for key, value in output.items():
        # Node
        pprint(f"Node '{key}':")
        # Optional: print full state at each node
        # pprint.pprint(value["keys"], indent=2, width=80, depth=None)
    pprint("\n---\n")

# Final generation
pprint(value["generation"])

---ROUTE QUESTION---


/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:390: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


---ROUTE QUESTION TO WEB SEARCH---
---WEB SEARCH---
"Node 'web_search':"
'\n---\n'
---GENERATE---


/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:390: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


The Chicago Bears drafted tight end Colston Loveland with the 10th overall pick in the 2025 NFL Draft. Loveland played college football at the University of Michigan. The Bears also drafted wide receiver Luther Burden out of Missouri with the No. 39 pick.
---CHECK HALLUCINATIONS---


/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:390: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---
---GRADE GENERATION vs QUESTION---


/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:390: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


---DECISION: GENERATION ADDRESSES QUESTION---
"Node 'generate':"
'\n---\n'
('The Chicago Bears drafted tight end Colston Loveland with the 10th overall '
 'pick in the 2025 NFL Draft. Loveland played college football at the '
 'University of Michigan. The Bears also drafted wide receiver Luther Burden '
 'out of Missouri with the No. 39 pick.')
